In [ ]:
import logging
from telegram import Update
from telegram.ext import Application, CommandHandler, MessageHandler, filters
# telegram.constants에서 HTML Parse Mode를 가져옵니다.
# from telegram.constants import ParseMode # 이 라인은 python-telegram-bot 버전 20.x 이후부터는 필요 없습니다.

# 1. 봇 토큰 설정
TOKEN = "8019568002:AAGr7DJEjevhDXGxoKX1D0QuC9PMUMlNmNA"

# 로깅 설정
logging.basicConfig(
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s", level=logging.INFO
)
logging.getLogger("httpx").setLevel(logging.WARNING)
logger = logging.getLogger(__name__)

# 2. 봇 명령어 핸들러 함수 정의
async def start(update: Update, context: object) -> None:
    """/start 명령어를 받았을 때 응답합니다."""
    user = update.effective_user

    # 1. 이미지 보내기 예시
    # 테스트용 이미지 URL (실제 사용할 이미지 URL로 변경하세요!)
    image_url = "https://cdn.imweb.me/upload/S20210103c5204f5ed12ce/8017dcc999129.png" # 예시: 파이썬 로고
    # image_url = "https://www.telegram.org/img/t_logo.png" # 예시: 텔레그램 로고

    # await context.bot.send_photo()를 사용하여 이미지를 보냅니다.
    await context.bot.send_photo(
        chat_id=update.effective_chat.id,
        photo=image_url,
        caption="저희 봇의 로고입니다!", # 이미지 캡션 (선택 사항)
    )
    logger.info(f"Sent image to user {user.first_name}.")

    # 2. 소개 페이지 (HTML 형식의 꾸며진 텍스트와 링크)
    introduction_text = f"""
안녕하세요, <b>{user.mention_html()}</b>님! 👋
저는 여러분의 <b>똑똑한 비서 봇</b>입니다.

이 봇은 다음과 같은 기능을 제공해요:
•  `/start`: 이 환영 메시지를 다시 봅니다.
•  일반 텍스트 메시지를 보내면 제가 <b>따라해 드려요!</b> (에코 기능)
•  <i>앞으로 더 많은 기능이 추가될 예정입니다!</i>

저에 대해 더 자세히 알고 싶으시면 <a href="https://www.example.com">여기</a>를 클릭해주세요.
궁금한 점이 있으시면 언제든지 메시지를 보내주세요!
    """

    # reply_html을 사용하여 HTML 형식으로 메시지를 보냅니다.
    # parse_mode="HTML"은 최신 python-telegram-bot 버전에서는 기본값이 되어 생략 가능하지만, 명시적으로 적는 것이 좋습니다.
    await update.message.reply_html(
        introduction_text,
        # parse_mode=ParseMode.HTML # 이 라인은 python-telegram-bot 버전 20.x 이후부터는 보통 생략 가능합니다.
    )
    logger.info(f"User {user.first_name} started the bot and received intro.")

# 나머지 echo, error_handler, main 함수는 동일합니다.
async def echo(update: Update, context: object) -> None:
    """사용자의 메시지를 그대로 다시 보냅니다."""
    await update.message.reply_text(update.message.text)
    logger.info(f"Echoed message: {update.message.text}")

async def error_handler(update: Update, context: object) -> None:
    """봇의 오류를 로깅합니다."""
    logger.warning(f"업데이트 {update}에서 오류가 발생했습니다: {context.error}")

def main() -> None:
    """봇을 실행합니다."""
    application = Application.builder().token(TOKEN).build()

    application.add_handler(CommandHandler("start", start))
    application.add_handler(MessageHandler(filters.TEXT & ~filters.COMMAND, echo))
    application.add_error_handler(error_handler)

    logger.info("봇을 시작합니다...")
    application.run_polling(allowed_updates=Update.ALL_TYPES)
    logger.info("봇이 종료되었습니다.")

if __name__ == "__main__":
    main()

import logging
import httpx  # API 호출을 위한 라이브러리 임포트
from telegram import Update
from telegram.ext import Application, CommandHandler, MessageHandler, filters

# 1. 봇 토큰 설정 (아까 BotFather에게서 받은 토큰을 여기에 붙여넣으세요!)
TOKEN = "8019568002:AAGr7DJEjevhDXGxoKX1D0QuC9PMUMlNmNA"

# 로깅 설정
logging.basicConfig(
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s", level=logging.INFO
)
logging.getLogger("httpx").setLevel(logging.WARNING)
logger = logging.getLogger(__name__)


# --- 새로운 부분: API 호출 함수 정의 ---
async def get_api_data() -> str:
    """
    외부 API를 호출하여 데이터를 가져오는 비동기 함수.
    이 함수는 예시이며, 실제 사용할 API에 맞게 수정해야 합니다.
    """
    # **중요:** 여기에 실제 사용할 API의 URL과 필요한 API 키를 넣으세요.
    # 이 예시는 OpenWeatherMap API의 대략적인 구조를 따릅니다.
    # 실제 API 키는 반드시 발급받아야 합니다!
    api_url = "https://api.openweathermap.org/data/2.5/weather"
    city_name = "Wonju"  # 원하는 도시 이름
    # **YOUR_OPENWEATHERMAP_API_KEY_HERE** 부분을 당신의 실제 API 키로 교체하세요!
    api_key = "6b00b6e4bb24cbe159d9ecda9252fcc3"

    params = {
        "q": city_name,
        "appid": api_key,
        "units": "metric",  # 섭씨 온도를 원하면 'metric', 화씨는 'imperial'
        "lang": "kr"  # 한국어 설명을 원하면 'kr'
    }

    async with httpx.AsyncClient() as client:
        try:
            response = await client.get(api_url, params=params, timeout=10)  # 10초 타임아웃
            response.raise_for_status()  # HTTP 오류 (4xx, 5xx)가 발생하면 예외 발생

            data = response.json()
            # API 응답에서 원하는 정보를 추출하여 문자열로 포맷팅합니다.
            # 이 부분은 사용하는 API의 응답 구조에 따라 달라집니다.

            # OpenWeatherMap 예시 응답 구조를 기반으로 데이터 추출
            weather_description = data['weather'][0]['description']
            temperature = data['main']['temp']
            feels_like = data['main']['feels_like']
            humidity = data['main']['humidity']

            return (f"현재 {city_name} 날씨 정보:\n"
                    f"✨ 날씨: {weather_description}\n"
                    f"🌡️ 온도: {temperature}°C (체감: {feels_like}°C)\n"
                    f"💧 습도: {humidity}%\n")

        except httpx.RequestError as e:
            logger.error(f"API 요청 중 오류 발생: {e}")
            return "날씨 정보를 가져오는 데 실패했습니다. 네트워크 연결을 확인하거나 API 키를 점검해주세요. 😞"
        except KeyError as e:
            logger.error(f"API 응답에서 키 오류 발생: {e}. 응답 구조가 예상과 다릅니다.")
            return "날씨 정보를 처리하는 데 문제가 발생했습니다. (데이터 형식 오류) 🧐"
        except Exception as e:
            logger.error(f"예상치 못한 오류 발생: {e}")
            return "날씨 정보를 가져오는 중 알 수 없는 오류가 발생했습니다. 🥺"


# --- 기존 코드 (start 함수만 수정) ---
async def start(update: Update, context: object) -> None:
    """/start 명령어를 받았을 때 응답합니다."""
    user = update.effective_user

    # 1. 이미지 보내기 예시 (이전과 동일)
    image_url = "https://cdn.imweb.me/upload/S20210103c5204f5ed12ce/8017dcc999129.png"
    await context.bot.send_photo(
        chat_id=update.effective_chat.id,
        photo=image_url,
        caption="저희 봇의 로고입니다!",
    )
    logger.info(f"Sent image to user {user.first_name}.")

    # --- 새로운 부분: API 호출하여 정보 가져오기 ---
    api_info_result = await get_api_data()  # 위에서 정의한 API 호출 함수 실행

    # 2. 소개 페이지 (HTML 형식의 꾸며진 텍스트와 링크)
    introduction_text = f"""
안녕하세요, <b>{user.mention_html()}</b>님! 👋
저는 미래채움 <b>봇</b>입니다.

{api_info_result} # API 호출 결과가 여기에 삽입됩니다!

이 봇은 다음과 같은 기능을 제공해요:
•  <code>/start</code>: 원주고 이미지와, 날씨 정보를 제공합니다
• 에코기능이 있어서 따라 따라말하는 기능이 있습니다.
•  <i>앞으로 더 많은 기능이 추가될 예정입니다 감사합니다</i>

    """

    await update.message.reply_html(introduction_text)
    logger.info(f"User {user.first_name} started the bot and received intro with API data.")


# 나머지 echo, error_handler, main 함수는 이전과 동일합니다.
async def echo(update: Update, context: object) -> None:
    """사용자의 메시지를 그대로 다시 보냅니다."""
    await update.message.reply_text(update.message.text)
    logger.info(f"Echoed message: {update.message.text}")


async def weather_command(update: Update, context: object) -> None:
    """/weather 명령어를 받았을 때 날씨 정보를 응답합니다."""
    user = update.effective_user
    logger.info(f"User {user.first_name} requested weather info.")

    # get_api_data() 함수를 호출하여 날씨 정보를 가져옵니다.
    weather_info = await get_api_data()

    # 가져온 날씨 정보를 사용자에게 메시지로 보냅니다.
    await update.message.reply_text(weather_info)


async def error_handler(update: Update, context: object) -> None:
    """봇의 오류를 로깅합니다."""
    logger.warning(f"업데이트 {update}에서 오류가 발생했습니다: {context.error}")


def main() -> None:
    """봇을 실행합니다."""
    application = Application.builder().token(TOKEN).build()

    application.add_handler(CommandHandler("start", start))
    application.add_handler(CommandHandler("weather", weather_command))  # <-- 이 줄 추가!

    application.add_handler(MessageHandler(filters.TEXT & ~filters.COMMAND, echo))
    application.add_error_handler(error_handler)

    logger.info("봇을 시작합니다...")
    application.run_polling(allowed_updates=Update.ALL_TYPES)
    logger.info("봇이 종료되었습니다.")


if __name__ == "__main__":
    main()